In [ ]:
# 1. Cài đặt thư viện
!pip install presidio-analyzer presidio-anonymizer faker pytesseract pdf2image
!apt-get install -y tesseract-ocr
!apt-get install -y poppler-utils
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
from faker import Faker

In [ ]:
#!pip install presidio-analyzer presidio-anonymizer faker python-docx transformers torch
#!python -m spacy download en_core_web_sm


!pip install presidio-analyzer presidio-anonymizer faker python-docx transformers torch spacy
!python -m spacy download fr_core_news_lg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.8/571.8 MB 873.0 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Code pour download

In [5]:
import pandas as pd
from docx import Document
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
from faker import Faker
import uuid
from google.colab import files


analyzer = AnalyzerEngine()
fake = Faker(['fr_FR', 'fr_FR'])

class PrivaShieldColab:
    def __init__(self):
        self.vault = {} # Mapping {Fake_Value: Real_Value}

    def smart_substitution(self, original_text, entity_type):
        """Tạo dữ liệu giả dựa trên loại thông tin nhạy cảm"""
        if entity_type == "PERSON":
            fake_val = fake.name()
        elif entity_type == "EMAIL_ADDRESS":
            fake_val = fake.email()
        elif entity_type == "PHONE_NUMBER":
            fake_val = "09" + fake.msisdn()[2:10]
        elif entity_type == "LOCATION":
            fake_val = fake.address().replace('\n', ', ')
        else:
            # Token
            fake_val = f"[{entity_type}_{uuid.uuid4().hex[:4].upper()}]"

        self.vault[fake_val] = original_text
        return fake_val

    def process_docx(self, file_path):
        doc = Document(file_path)
        total_paragraphs = len(doc.paragraphs)
        print(f"--- processed {total_paragraphs} paragraph ---")

        for i, para in enumerate(doc.paragraphs):
            if len(para.text.strip()) > 0:
                # Analyse PII
                results = analyzer.analyze(text=para.text, entities=[], language='en')

                #
                sorted_results = sorted(results, key=lambda x: x.start, reverse=True)

                new_para_text = para.text
                for res in sorted_results:
                    original_val = para.text[res.start:res.end]
                    fake_val = self.smart_substitution(original_val, res.entity_type)
                    new_para_text = new_para_text[:res.start] + fake_val + new_para_text[res.end:]

                para.text = new_para_text

            if i % 10 == 0:
                print(f"Fini procédure: {i}/{total_paragraphs}...")


        output_name = "sanitized_" + file_path
        doc.save(output_name)
        return output_name

    def export_vault(self):
        """Xuất file Excel Mapping"""
        df = pd.DataFrame(list(self.vault.items()), columns=['Fake_Data', 'Original_Data'])
        df.to_csv("mapping_vault.csv", index=False)
        return "mapping_vault.csv"

priva = PrivaShieldColab()

print("Choisir fichier .docx Note Fiscale):")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

output_file = priva.process_docx(file_name)
vault_file = priva.export_vault()

print("\n--- Process terminé ---")
files.download(output_file)
files.download(vault_file)

Choisir fichier .docx Note Fiscale):


Saving SARL GLOBALSILVER-memorandum fiscal-12(1).docx to SARL GLOBALSILVER-memorandum fiscal-12(1) (4).docx
--- processed 835 paragraph ---
Fini procédure: 0/835...
Fini procédure: 10/835...
Fini procédure: 20/835...
Fini procédure: 30/835...
Fini procédure: 40/835...
Fini procédure: 50/835...
Fini procédure: 60/835...
Fini procédure: 70/835...
Fini procédure: 80/835...
Fini procédure: 90/835...
Fini procédure: 100/835...
Fini procédure: 110/835...
Fini procédure: 120/835...
Fini procédure: 130/835...
Fini procédure: 140/835...
Fini procédure: 150/835...
Fini procédure: 160/835...
Fini procédure: 170/835...
Fini procédure: 180/835...
Fini procédure: 190/835...
Fini procédure: 200/835...
Fini procédure: 210/835...
Fini procédure: 220/835...
Fini procédure: 230/835...
Fini procédure: 240/835...
Fini procédure: 250/835...
Fini procédure: 260/835...
Fini procédure: 270/835...
Fini procédure: 280/835...
Fini procédure: 290/835...
Fini procédure: 300/835...
Fini procédure: 310/835...
Fini pr

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

###Spacy

In [2]:
# 1️⃣ Installer les librairies
!pip install presidio-analyzer presidio-anonymizer faker python-docx transformers torch spacy
!python -m spacy download fr_core_news_lg

import pandas as pd
from docx import Document
from presidio_analyzer import AnalyzerEngine, PatternRecognizer, Pattern
from faker import Faker
import uuid
import re
from google.colab import files
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# 2️⃣ Configurer Faker et analyzer
fake = Faker(['fr_FR'])
analyzer = AnalyzerEngine(default_score_threshold=0.3)

# 3️⃣ Regex custom pour SIRET, IBAN, VAT, ZIP
vat_pattern = Pattern(name="vat_pattern", regex=r"(VAT|TVA|number)[:\s]*([A-Z]{0,2}\s?\d{8,11})", score=0.8)
vat_recognizer = PatternRecognizer(supported_entity="VAT_NUMBER", patterns=[vat_pattern])

siret_pattern = Pattern(name="siret_pattern", regex=r"(SIRET|SIREN|Registered)[:\s]*(\d{8,14})", score=0.8)
siret_recognizer = PatternRecognizer(supported_entity="SIRET", patterns=[siret_pattern])

zip_pattern = Pattern(name="zip_pattern", regex=r"\b([A-Z]{1,2}\d[A-Z\d]?\s\d[A-Z]{2}|[0-9]{5})\b", score=0.8)
zip_recognizer = PatternRecognizer(supported_entity="ZIP_CODE", patterns=[zip_pattern])

# Ajouter les recognizers
analyzer.registry.add_recognizer(vat_recognizer)
analyzer.registry.add_recognizer(siret_recognizer)
analyzer.registry.add_recognizer(zip_recognizer)

# 4️⃣ Charger CamemBERT NER pour PII
ner_model_name = "Jean-Baptiste/camembert-ner-with-dates"
tokenizer = AutoTokenizer.from_pretrained(ner_model_name, use_fast=True)
model = AutoModelForTokenClassification.from_pretrained(ner_model_name)
ner = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# 5️⃣ Classe anonymisation
class PrivaShieldFrance:
    def __init__(self):
        self.vault = {}

    def fake_value(self, entity, original_text):
        """Génère une valeur fake selon type d'entité"""
        if entity == "PERSON":
            val = fake.name()
        elif entity == "LOCATION":
            val = fake.address().replace("\n", ", ")
        elif entity == "ORGANIZATION":
            val = fake.company()
        elif entity == "SIRET":
            val = fake.siret()
        elif entity == "VAT_NUMBER":
            val = "FR" + str(fake.random_number(digits=11))
        elif entity == "ZIP_CODE":
            val = fake.postcode()
        elif entity == "IBAN":
            val = fake.iban()
        elif entity == "EMAIL":
            val = fake.email()
        else:
            val = f"[{entity}_{uuid.uuid4().hex[:4].upper()}]"

        self.vault[val] = original_text
        return val

    def process_docx(self, file_path):
        doc = Document(file_path)
        for para in doc.paragraphs:
            if para.text.strip():
                # 1️⃣ Regex + Presidio
                regex_results = analyzer.analyze(text=para.text,language='fr')
                # 2️⃣ NER CamemBERT
                ner_results = ner(para.text)
                # Fusionner résultats
                spans = [{"start": r.start, "end": r.end, "entity": r.entity_type, "text": para.text[r.start:r.end]} for r in regex_results]
                spans += [{"start": e['start'], "end": e['end'], "entity": e['entity_group'], "text": e['word']} for e in ner_results]

                spans_sorted = sorted(spans, key=lambda x: x["start"], reverse=True)
                new_text = para.text
                for s in spans_sorted:
                    new_text = new_text[:s["start"]] + self.fake_value(s["entity"], s["text"]) + new_text[s["end"]:]
                para.text = new_text

        output_name = "RGPD_Protected_" + file_path
        doc.save(output_name)
        return output_name

# 6️⃣ Upload et traitement
priva = PrivaShieldFrance()
print("Upload votre Note Fiscale (.docx):")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

output_file = priva.process_docx(file_name)

# 7️⃣ Export mapping
df_vault = pd.DataFrame(list(priva.vault.items()), columns=["Fake_Data", "Original_Data"])
df_vault.to_csv("mapping_vault.csv", index=False)

# 8️⃣ Download fichiers
files.download(output_file)
files.download("mapping_vault.csv")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.8/571.8 MB 777.2 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Device set to use cpu


Upload votre Note Fiscale (.docx):


Saving SARL GLOBALSILVER-memorandum fiscal-12(1).docx to SARL GLOBALSILVER-memorandum fiscal-12(1) (1).docx


ValueError: No matching recognizers were found to serve the request.